In [14]:
import pandas as pd
import numpy as np

In [15]:
# open elections_with_tracts.xlsx
elections_with_tracts = pd.read_excel('elections_with_tracts.xlsx')
# open acs_data.csv
acs_data = pd.read_csv('acs_data.csv')
# remove all columns ending in EA
acs_data = acs_data.loc[:, ~acs_data.columns.str.endswith('EA')]
# replace all (X), -, or N with nan
acs_data = acs_data.replace('(X)', 0)
acs_data = acs_data.replace('-', 0)
acs_data = acs_data.replace('N', 0)
# convert all columns except the first 2 to floats
for col in acs_data.columns[2:]:
    acs_data[col] = acs_data[col].astype(float)
    # replace all negative values with NaN except the first 4 columns
    acs_data[col] = acs_data[col].apply(lambda x: 0 if x < 0 else x)
acs_data

C:\Users\User\AppData\Local\Temp\ipykernel_3792\431352590.py:4: DtypeWarning: Columns (3,21,23,30,33,48,60,61,64,71,88,134,154,155,168,172,182,205,206,216,218,221,222,246,252,263,264,268,274,277,282,288,291,293,295,304,307,308,314,323,337,350,381,386,415,456,462,488,544,548,557,560,562,567,578,579,581,600,610,611,630,635,645,648,655,664,678,684,699,704,709,710,724,732,736,744,750) have mixed types. Specify dtype option on import or set low_memory=False.
  acs_data = pd.read_csv('acs_data.csv')


,geocode,year,DP02_0127PE,DP02_0039PE,DP02_0130E,DP03_0051E,DP03_0010E,DP03_0121E,DP02_0069PE,DP02_0090E,...,DP03_0115E,DP02_0124PE,DP05_0008PE,DP02_0154PE,DP02_0062E,DP03_0067E,DP05_0003E,DP02_0033PE,DP02_0046PE,DP03_0063E
0,06_001_422200,2010,2.6,0.0,13.0,1522.0,1506.0,0.0,3.3,1044.0,...,318.0,1.3,8.4,NaN,228.0,15020.0,1763.0,2.0,0.0,79904.0
1,06_001_422400,2010,1.6,0.0,0.0,2171.0,1597.0,0.0,1.3,1095.0,...,1425.0,0.3,32.4,NaN,353.0,14688.0,1691.0,0.0,0.0,66278.0
2,06_001_422300,2010,2.4,0.0,10.0,1591.0,1488.0,0.0,5.3,1198.0,...,466.0,1.1,17.9,NaN,392.0,12681.0,1781.0,1.7,0.0,77234.0
3,06_001_420400,2010,0.0,0.0,8.0,1160.0,1238.0,0.0,1.1,487.0,...,965.0,2.1,2.5,NaN,148.0,12903.0,1777.0,5.5,0.0,51303.0
4,06_001_421800,2010,1.4,0.0,0.0,897.0,1025.0,0.0,5.3,784.0,...,213.0,0.0,5.1,NaN,177.0,18863.0,1180.0,0.5,0.0,110333.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6497,02_170_000101,2022,1.4,0.0,28.0,270.0,194.0,0.0,470.0,499.0,...,72.0,512.0,2.7,74.8,200.0,18418.0,213.0,64.4,0.0,77947.0
6498,02_050_000200,2022,0.1,0.0,141.0,1937.0,2058.0,0.0,4403.0,5919.0,...,526.0,6312.0,7.9,91.8,1622.0,18060.0,2878.0,41.9,0.0,118182.0
6499,50_007_003600,2023,0.3,0.0,896.0,2511.0,2624.0,0.0,4448.0,4255.0,...,374.0,4990.0,2.1,79.3,673.0,19693.0,2928.0,25.4,0.0,78025.0
6500,50_007_000800,2023,0.6,0.0,687.0,1032.0,1130.0,0.0,1962.0,2331.0,...,193.0,2430.0,2.8,96.9,180.0,27227.0,1276.0,37.3,0.0,156711.0


In [16]:
# open variables/variables_acs5_2022.csv
variables = pd.read_csv('variables/variables_acs5_2022.csv')

percent_columns = variables[variables['Label'].str.lower().str.contains(r'\brate\b|\bpercentage\b')]
percent_columns['Name'] = percent_columns['Name'].str[:-2]
# for each row in acs_data
for index, row in acs_data.iterrows():
    # for each column in percent_columns
    for col in percent_columns['Name']:
        # if col + E and col + PE are in acs columns
        if col + 'E' in acs_data.columns and col + 'PE' in acs_data.columns:
            # get col + E and col + PE
            est = row[col + 'E']
            percent_est = row[col + 'PE']

            # if percent_est is nan, set PE to est and set E to nan
            if np.isnan(percent_est) or percent_est == 0:
                acs_data.at[index, col + 'PE'] = est
                acs_data.at[index, col + 'E'] = np.nan

C:\Users\User\AppData\Local\Temp\ipykernel_3792\2253298472.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  percent_columns['Name'] = percent_columns['Name'].str[:-2]


In [17]:
# checking for columns with invalid data
invalid = []
# for each column except the first 2
for col in acs_data.columns[2:]:
    valid = False
    # for each value in the column
    for val in acs_data[col]:
        # if the value is not nan and is greater than 0
        if not np.isnan(val) and val > 0:
            valid = True
            break
    if not valid:
        invalid.append(col)
# remove invalid columns
acs_data = acs_data.drop(columns=invalid)

In [18]:
# for each column in acs_data
for col in acs_data.columns:
   # if it ends in E and not PE
    if col.endswith('E') and not col.endswith('PE'):
        # if every value in the column is not nan
        if acs_data[col].isnull().sum() == 0:
            # remove the PE column
            acs_data = acs_data.drop(columns=[col[:-1] + 'PE'])

In [19]:
# open statistics.xlsx
statistics = pd.read_excel('statistics.xlsx')
statistics

,agg_var,type,pop_var
0,DP03_0119PE,percentage,DP03_0075E
1,DP03_0120PE,percentage,DP02_0014E
2,DP03_0121PE,percentage,NaN
3,DP03_0122PE,percentage,DP02_0002E
4,DP03_0123PE,percentage,DP02_0003E
5,DP03_0124PE,percentage,NaN
6,DP03_0125PE,percentage,DP02_0010E
7,DP03_0126PE,percentage,DP02_0011E
8,DP03_0127PE,percentage,NaN
9,DP03_0128PE,percentage,DP05_0001E


In [20]:
# open elections.xlsx
elections = pd.read_excel('elections.xlsx')
elections

,filename,city,state,Geography,District #,office,year,geo_filename
0,Berkeley_11022010_CityCouncilDistrict1.csv,Berkeley,CA,city council,1,City council,2010,berkeley_city_council_2002.csv
1,Berkeley_11042014_CityCouncilDistrict1.csv,Berkeley,CA,city council,1,City council,2014,berkeley_city_council_2014.csv
2,Berkeley_11062018_CityCouncilDistrict1.csv,Berkeley,CA,city council,1,City council,2018,berkeley_city_council_2014.csv
3,Minneapolis_11022021_CityCouncilWard1.csv,Minneapolis,MN,city council,1,City council,2021,minneapolis_city_council_2012.csv
4,Minneapolis_11052013_Ward13CityCouncil.csv,Minneapolis,MN,city council,1,City council,2013,minneapolis_city_council_2012.csv
...,...,...,...,...,...,...,...,...
494,Cambridge_11062007_SchoolCommittee.csv,Cambridge,MA,school board,NaN,School committee,2007,NaN
495,Cambridge_11052013_SchoolCommittee.csv,Cambridge,MA,school board,NaN,School committee,2013,NaN
496,Cambridge_11072017_SchoolCommittee.csv,Cambridge,MA,school board,NaN,School committee,2017,NaN
497,SanFrancisco_11052019_Sheriff.csv,San Francisco,CA,city,NaN,Sheriff,2019,NaN


In [21]:
all_data = []

# create df called missing_data with columns filename, year, values, num_missing, num_total, missing_tracts
missing_data = pd.DataFrame(columns=['filename', 'year', 'variable', 'values', 'num_missing', 'num_total', 'prop_missing'])

for index, row in elections_with_tracts.iterrows():
    if index % 10 == 0:
        print(f'{index} of {len(elections_with_tracts)} rows processed')

    tracts = row['tracts'].split(',')
    # year is year in row in elections where filename = row['filename']
    year = elections[elections['filename'] == row['filename']]['year'].values[0]
    # get rows where geocode is in tracts and year = year
    tract_rows = acs_data[(acs_data['geocode'].isin(tracts)) & (acs_data['year'] == year)]

    # convert row['percents'] to list of floats
    percents = list(map(float, row['percents'].split(',')))
    election_data = {}

    for column in acs_data.columns[2:]:
        # get the values in tract_rows for column
        values = tract_rows[column].values

        # if all values are NaN, continue
        if all(np.isnan(values)):
            continue

        # if at least one but not all values are NaN, print values
        if any(np.isnan(values)):
            # add to missing_data
            missing_data.loc[len(missing_data)] = [row['filename'], year, column, values, sum(np.isnan(values)), len(values), sum(np.isnan(values)) / len(values)]
            
        if column in statistics['agg_var'].values:
            pass
        else:
            pass
    

    all_data.append(election_data)

0 of 356 rows processed
10 of 356 rows processed
20 of 356 rows processed
30 of 356 rows processed
40 of 356 rows processed
50 of 356 rows processed
60 of 356 rows processed
70 of 356 rows processed
80 of 356 rows processed
90 of 356 rows processed
100 of 356 rows processed
110 of 356 rows processed
120 of 356 rows processed
130 of 356 rows processed
140 of 356 rows processed
150 of 356 rows processed
160 of 356 rows processed
170 of 356 rows processed
180 of 356 rows processed
190 of 356 rows processed
200 of 356 rows processed
210 of 356 rows processed
220 of 356 rows processed
230 of 356 rows processed
240 of 356 rows processed
250 of 356 rows processed
260 of 356 rows processed
270 of 356 rows processed
280 of 356 rows processed
290 of 356 rows processed
300 of 356 rows processed
310 of 356 rows processed
320 of 356 rows processed
330 of 356 rows processed
340 of 356 rows processed
350 of 356 rows processed


In [22]:
# save missing_data to csv
missing_data.to_csv('missing_data.csv', index=False)

In [23]:
# all_data_df = pd.DataFrame(all_data)
# result_df = pd.concat([elections_with_tracts, all_data_df], axis=1)
# result_df